#**Libraries**

#**Feature Snipping**

In [ ]:
def upload_cropped_image(img_cropped, folder_name, img_name):
  folder_path = f'/content/drive/My Drive/AgriLifeMicrohistologicalProject/FeatureImages/{folder_name}'#path
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

  fig, ax = plt.subplots()  # Create a figure and axes
  ax.imshow(img_cropped)  # Display the image on the axes
  ax.axis('off')  # Turn off axis markers

  # Save the figure, if there's a duplicate save with the same name savefig will auto overwrite the previous save so no duplicates
  fig.savefig(f'{folder_path}/{img_name}.png', bbox_inches='tight', pad_inches = 0)
  plt.close(fig)

In [ ]:
#colab can't access shared folders so I created a folder in my drive and to add individual folders from the data
#reading text file
with open('drive/MyDrive/AgriLifeMicrohistologicalProject/TrainingDataset/136 Export/004/136 Export004.txt', newline='') as f: #reads txt file and turns into 2d list
    reader = csv.reader(f, delimiter=' ')
    data = list(reader)

grass = [] #grass/holder lists
holder = []
count = 0

for x in data: #turns all string in 2d list to float
  for y in x:
    holder.append(float(y))
    count += 1

  if count == 5:
    count = 0
    grass.append(holder)#appends to grass list
    holder = []

print(grass)

[[2.0, 0.567773, 0.091004, 0.065137, 0.07034], [3.0, 0.567871, 0.189835, 0.02373, 0.027865], [2.0, 0.700171, 0.143916, 0.0521, 0.06053], [1.0, 0.257642, 0.275047, 0.050439, 0.03757], [1.0, 0.939624, 0.042502, 0.056982, 0.074776]]


In [ ]:
img_path = 'drive/MyDrive/AgriLifeMicrohistologicalProject/TrainingDataset/136 Export/004/136 Export004.png'

#creates concatenates to name the files based on the given unique image name
filename = os.path.basename(img_path).split('.')[0].replace(" ", "")

img = plt.imread(img_path) #gets image data

nale_count = 0 #variable counts
qufu_count = 0
erci_count = 0
bubble_count = 0
qufu_stem_count = 0

#crops the image based on every line in the text file and uploads them to a given folder using each picture's unique name
for ind in range(len(grass)):
  height, width = img.shape[0:2] #stores dimensions of image

  #snipping math using txt file numbers
  x_min = round((width * grass[ind][1]) - (width * grass[ind][3])/2)
  x_max = round((width * grass[ind][1]) + (width * grass[ind][3])/2)
  y_min = round((height * grass[ind][2]) - (height * grass[ind][4])/2)
  y_max = round((height * grass[ind][2]) + (height * grass[ind][4])/2)

  img_cropped = img[y_min:y_max, x_min:x_max]
  if(grass[ind][0] == 0):
    nale_count += 1 #labels each 5 unique feature snipped from the image
    upload_cropped_image(img_cropped, '0 - nale', f"{filename}_{nale_count}")

  elif(grass[ind][0] == 1):
    qufu_count += 1
    upload_cropped_image(img_cropped, '1 - qufu', f"{filename}_{qufu_count}")

  elif(grass[ind][0] == 2):
    erci_count += 1
    upload_cropped_image(img_cropped, '2 - erci', f"{filename}_{erci_count}")

  elif(grass[ind][0] == 3):
    bubble_count += 1
    upload_cropped_image(img_cropped, '3 - bubble', f"{filename}_{bubble_count}")

  #elif(grass[ind][0] == 4):
   # qufu_stem_count += 1
   # upload_cropped_image(img_cropped, '4 - qufu stem', f"{filename}_{qufu_stem_count}")

#**Sub Feature Snipping**

In [7]:
# Get a list of all featimg files in the '4 - qufu stem' folder
featfolder = 'test'
featimg_folder = f'drive/MyDrive/AgriLifeMicrohistologicalProject/FeatureImages/{featfolder}'
featimg_files = [f for f in os.listdir(featimg_folder) if f.endswith('.png')]#all images

# Splice each featimg
output_subimg_folder = f'drive/MyDrive/AgriLifeMicrohistologicalProject/SubImages/{featfolder}'
for featimg_file in featimg_files:
    featimg_path = os.path.join(featimg_folder, featimg_file) #creates the feature path
    splice_featimg(featimg_path, output_subimg_folder)

136Export001_1_sub_0 whitespace percent: 72.94
136Export001_1_sub_1 whitespace percent: 41.449999999999996
136Export001_1_sub_2 whitespace percent: 62.019999999999996
136Export001_1_sub_3 whitespace percent: 82.17
136Export001_1_sub_4 whitespace percent: 21.4
136Export001_1_sub_5 whitespace percent: 0.0
136Export001_1_sub_6 whitespace percent: 0.0
136Export001_1_sub_7 whitespace percent: 0.0
136Export001_1_sub_8 whitespace percent: 86.14
136Export001_1_sub_9 whitespace percent: 70.19
136Export001_1_sub_10 whitespace percent: 48.59
136Export001_1_sub_11 whitespace percent: 27.860000000000003


In [5]:
def splice_featimg(featimg_path, output_folder, sub_image_size=100):
    featimg = plt.imread(featimg_path)

    # Get image dimensions
    height, width, channels = featimg.shape
    #print(featimg.shape)

    # Iterate over the image in sub-image-sized blocks
    sub_image_count = 0  # Initialize sub-image counter
    for y in range(0, height, sub_image_size):
        for x in range(0, width, sub_image_size):
            # Extract the sub-image
            sub_img = featimg[y:y + sub_image_size, x:x + sub_image_size]

            #checks if sub image is the expected dimensions and checks to make sure it's not all whitespace
            if sub_img.shape[0] == sub_image_size and sub_img.shape[1] == sub_image_size:

                # Separate the RGB and alpha channels
                #print(sub_img.shape)
                rgb = sub_img[..., :3]
                alpha = sub_img[..., 3]

                # Convert the RGB part to grayscale
                grayscale = np.mean(rgb, axis=-1)

                # Apply the threshold to detect white pixels
                white_pixels = np.sum((grayscale >= (240/255)) & (alpha > 0))

                # Total number of non-transparent pixels
                total_pixels = np.sum(alpha > 0)

                # Calculate the percentage of white pixels
                whitespace_percentage = (white_pixels / total_pixels) * 100

                print(f"{os.path.basename(featimg_path).split('.')[0]}_sub_{sub_image_count} whitespace percent: {whitespace_percentage}")

                # Ignore sub-images with more than 95% whitespace
                if whitespace_percentage < 95:

                  # Create a figure and axes for the sub-image
                  fig, ax = plt.subplots()

                  # Display the sub-image on the axes
                  ax.imshow(sub_img)
                  ax.axis('off')  # Turn off axis markers

                  # Save the sub-image using plt.savefig and names it
                  sub_image_filename = f"{os.path.basename(featimg_path).split('.')[0]}_sub_{sub_image_count}.png"
                  sub_image_path = os.path.join(output_folder, sub_image_filename)
                  fig.savefig(sub_image_path, bbox_inches='tight', pad_inches=0)

                  # Close the figure to release resources
                  plt.close(fig)

                  sub_image_count += 1  # Increment sub-image counter

#**New Image Splitter**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import os
from PIL import Image
import numpy as np #import

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
img_path = 'drive/MyDrive/AgriLifeMicrohistologicalProject/NewImgDataset/184HD/001/184HD_001_RESIZE.png' #new image path
filename = os.path.basename(img_path).split('.')[0].replace(" ", "")
img = plt.imread(img_path) #reading image

output_folder = 'drive/MyDrive/AgriLifeMicrohistologicalProject/NewImages/new' #output path

Newimgsplice(img, filename, output_folder)

Streaming output truncated to the last 5000 lines.
184HD_001_RESIZE_1893 whitespace percent: 85.73621510626805
184HD_001_RESIZE_1894 whitespace percent: 94.09731481752581
184HD_001_RESIZE_1895 whitespace percent: 95.21547249848588
184HD_001_RESIZE_1895 whitespace percent: 98.66915394948751
184HD_001_RESIZE_1895 whitespace percent: 97.15777767968441
184HD_001_RESIZE_1895 whitespace percent: 95.07892293407613
184HD_001_RESIZE_1895 whitespace percent: 90.01408080383348
184HD_001_RESIZE_1896 whitespace percent: 87.20483143566226
184HD_001_RESIZE_1897 whitespace percent: 84.87474283968423
184HD_001_RESIZE_1898 whitespace percent: 68.3523598413483
184HD_001_RESIZE_1899 whitespace percent: 70.09074648449544
184HD_001_RESIZE_1900 whitespace percent: 70.74815951542357
184HD_001_RESIZE_1901 whitespace percent: 72.43561871272546
184HD_001_RESIZE_1902 whitespace percent: 79.56763306217096
184HD_001_RESIZE_1903 whitespace percent: 100.0
184HD_001_RESIZE_1903 whitespace percent: 76.08710440134226
18

In [2]:
def Newimgsplice(img, filename, output_folder, sub_image_size=100):

  height, width, channels = img.shape  # Get image dimensions
  print(img.shape)

  # Starting y position (halfway point)
  start_y = height // 2
  start_x = width // 2

  # Iterate over the image in sub-image-sized blocks along the x-axis only
  sub_image_count = 0  # Initialize sub-image counter
  for y in range(start_y, height, sub_image_size):
    for x in range(start_x, width, sub_image_size):
      # Extract the sub-image
      sub_img = img[y:y + sub_image_size, x:x + sub_image_size]

      # Check if sub-image has the expected dimensions
      if sub_img.shape[0] == sub_image_size and sub_img.shape[1] == sub_image_size:
        # Separate the RGB and alpha channels
        #print(sub_img.shape)
        rgb = sub_img[..., :3]

        # Convert the RGB part to grayscale
        grayscale = np.mean(rgb, axis=-1)

        # Apply the threshold to detect white pixels
        white_pixels = np.sum(grayscale >= (240/255))

        # Total number of pixels
        total_pixels = np.sum(grayscale)

        # Calculate the percentage of white pixels
        whitespace_percentage = (white_pixels / total_pixels) * 100

        print(f"{filename}_{sub_image_count} whitespace percent: {whitespace_percentage}")

        # Ignore sub-images with more than 95% whitespace
        if whitespace_percentage < 95:
          fig, ax = plt.subplots()
          ax.imshow(sub_img)
          ax.axis('off')  # Turn off axis markers

          # Save the sub-image and name it
          sub_image_filename = f"{filename}_{sub_image_count}.png"
          sub_image_path = os.path.join(output_folder, sub_image_filename)
          fig.savefig(sub_image_path, bbox_inches='tight', pad_inches=0)
          plt.close(fig)

          sub_image_count += 1  # Increment sub-image counter